베이즈 네트워크의 근사 추론(approximate inference). 여기 제공하는 코드는 GitHub aima-python의 코드를 기반으로 일부 수정한 것임.

In [ ]:
# 베이즈 네트워크 기본 코드를 bayesnet.py에 저장해뒀음
from bayesnet import *
seed = 23

## 근사 추론 알고리즘 구현

In [ ]:
# 무증거 랜덤 샘플링
def prior_sample(bn):
    """
    베이즈 네트워크 bn의 완전 결합 확률 분포에서 무작위 샘플링.
    {변수: 값} 형식의 딕셔너리가 리턴됨.
    """
    event = {}
    for node in bn.nodes:
        event[node.variable] = node.sample(event)
    return event

In [ ]:
# 기각 샘플링
def rejection_sampling(X, e, bn, N=10000):
    """
    N개의 샘플을 사용하여 P(X|e)를 추정함.
    N개 샘플이 모두 기각되면 ZeroDivisionError 발생.
    """
    counts = {x: 0 for x in bn.variable_values(X)}
    for j in range(N):
        sample = prior_sample(bn)
        if consistent_with(sample, e):
            counts[sample[X]] += 1
    return ProbDist(X, counts)


def consistent_with(event, evidence):
    """event가 evidence와 일관되는가?(증거와 부합하는가?)"""
    return all(evidence.get(k, v) == v for k, v in event.items())

In [ ]:
# 가능도 가중치
def likelihood_weighting(X, e, bn, N=10000):
    """
    N개의 샘플을 사용하여 P(X|e)를 추정함.
    """
    W = {x: 0 for x in bn.variable_values(X)}
    for j in range(N):
        sample, weight = weighted_sample(bn, e)
        W[sample[X]] += weight
    return ProbDist(X, W)


def weighted_sample(bn, e):
    """
    증거 e와 일관된 이벤트를 bn으로부터 샘플링함.
    이벤트와 가중치(이벤트가 증거에 부합할 가능도)를 리턴함.
    """
    w = 1
    event = dict(e)
    for node in bn.nodes:
        Xi = node.variable
        if Xi in e:
            w *= node.p(e[Xi], event)
        else:
            event[Xi] = node.sample(event)
    return event, w


In [ ]:
# Gibbs 샘플링
def gibbs_ask(X, e, bn, N=1000):
    """Gibbs 샘플링."""
    assert X not in e, "Query variable must be distinct from evidence"
    counts = {x: 0 for x in bn.variable_values(X)}
    Z = [var for var in bn.variables if var not in e]
    state = dict(e)
    for Zi in Z:
        state[Zi] = random.choice(bn.variable_values(Zi))
    for j in range(N):
        for Zi in Z:
            state[Zi] = markov_blanket_sample(Zi, state, bn)
            counts[state[X]] += 1
    return ProbDist(X, counts)


def markov_blanket_sample(X, e, bn):
    """
    P(X | mb)에 따라 샘플을 리턴함.
    mb: X의 마르코프 블랭킷에 속한 변수들. 그 값은 이벤트 e에서 취함.
    x의 마르코프 블랭킷: X의 부모들, 자식들, 자식들의 부모들
    """
    Xnode = bn.variable_node(X)
    Q = ProbDist(X)
    for xi in bn.variable_values(X):
        ei = extend(e, X, xi)
        Q[xi] = Xnode.p(xi, e) * product(Yj.p(ei[Yj.variable], ei) for Yj in Xnode.children)
    # 불리언 변수로 가정함.
    return probability(Q.normalize()[True])


def product(numbers):
    """numbers에 있는 값들을 모두 곱한 결과를 리턴함. 예: product([2, 3, 10]) == 60"""
    result = 1
    for x in numbers:
        result *= x
    return result

## 근사 추론 예

In [ ]:
# Sprinkler 베이즈 네트워크 생성
sprinkler = BayesNet([
    ('Cloudy', '', 0.5),
    ('Sprinkler', 'Cloudy', {T: 0.10, F: 0.50}),
    ('Rain', 'Cloudy', {T: 0.80, F: 0.20}),
    ('WetGrass', 'Sprinkler Rain', {(T, T): 0.99, (T, F): 0.90, (F, T): 0.90, (F, F): 0.00})])

In [ ]:
N = 1000
all_observations = [prior_sample(sprinkler) for x in range(N)]
all_observations[:5]

In [ ]:
# P(Rain=True)
rain_true = [observation for observation in all_observations if observation['Rain'] == True]
answer = len(rain_true) / N
print(answer)

In [ ]:
# 다시 샘플링하여 계산
N = 1000
all_observations = [prior_sample(sprinkler) for x in range(N)]
rain_true = [observation for observation in all_observations if observation['Rain'] == True]
answer = len(rain_true) / N
print(answer)

In [ ]:
# P(Cloudy=True | Rain=True)
rain_and_cloudy = [observation for observation in rain_true if observation['Cloudy'] == True]
answer = len(rain_and_cloudy) / len(rain_true)
print(answer)

In [ ]:
# 기각 샘플링을 사용하여 P(Cloudy=True | Rain=True) 추정
random.seed(seed)
dist = rejection_sampling('Cloudy', dict(Rain=True), sprinkler, 1000)
print(dist.show_approx())
print(dist[True])

In [ ]:
weighted_sample(sprinkler, dict(Rain=True))

In [ ]:
# 가능도 가중치를 사용하여 P(Cloudy=True | Rain=True) 추정
random.seed(seed)
dist = likelihood_weighting('Cloudy', dict(Rain=True), sprinkler, 1000)
print(dist.show_approx())
print(dist[True])

In [ ]:
# Gibbs 샘플링을 사용하여 P(Cloudy=True | Rain=True) 추정
dist = gibbs_ask('Cloudy', dict(Rain=True), sprinkler, 1000)
print(dist.show_approx())
print(dist[True])